# EXPECTING THE UNEXPECTED

Systems built with software can be fragile. We need to have a way to work around the spectrum of failures that plague computer systems.

We will study **exceptions**, special error objects raised when a normal response is impossible.



# RAISING EXCEPTIONS

Python's normal behaviour is to execute statements in the order they are found.

A few statements, specifically `if`, `while`, and `for`, alter the simple top-to-bottom sequence of statement execution.

Additionally, an exception can break the sequential flow of execution.

Exceptions are raised, and this interrupts the sequential execution of statements.

In Python, the exception that's raised is also an object. There are many different exception classes available and we can easily define more of our own.

The one thing that all exceptions have in common is that they are instances of a class that inherits from the `BaseException` class.

When an exception is raised, everything that was supposed to happen is pre-empted.

Instead, exception handling replaces the normal processing.

The easiest way to cause an exception to occur is to do something silly. 

For example, any time Python encounters a line in your program that it can't understand, it bails with SyntaxError, which is a type of exception.



In [3]:
print "Hello, world!"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2804745302.py, line 1)

The print() function requires the arguments to be enclosed in parentheses. So, if we type the preceding command into a Python 3 interpreter, we raise a SyntaxError exception.

We can partition exceptions into roughly four categories. Some cases are blurry, but some edges have a bright line separating them:

Sometimes, exceptions are indicators of something clearly wrong in our program. Exceptions like SyntaxError and NameError mean we need to find the indicated line number and fix the problem.


Sometimes, exceptions are indicators of something wrong in the Python runtime. There's a RuntimeError exception that can get raised. In many cases, this is resolved by downloading and installing a newer Python. 

Some exceptions are design problems. We may fail to account for an edge case properly and sometimes try to compute an average of an empty list. This will result in a `ZeroDivisionError`. When we find these, again, we'll have to go to the indicated line number. But once we've found the resulting exception, we'll need to work backwards from there to find out what caused the problem that raised the exception. 

# EFFECTS OF EXCEPTIONS

When an exception is raised, it appears to stop the program execution immediately.

Any lines that were supposed to run after the exception is raised are not executed unless the exception is handled by an `except` clause, the program will exit with an error message.

We can control how exceptions propagate from the initial `raise` statement. We can react to and deal with the exception inside either of these methods in the call stack.

# HANDLING EXCEPTIONS

If we encounter an exception situation, how should our code react to or recover from it? 

We handle exceptions by wrapping any code that might throw one (whether it is exception code itself, or a call to any function or method that may have an exception raised inside it) inside a `try...except` clause. 

The most basic syntax looks like this:

In [4]:
from typing import NoReturn
def never_returns() -> NoReturn:
    print("I am about to raise an exception")
    raise Exception("This is always raised")
    print("This line will never execute")
    return "I won't be returned"

In [5]:
never_returns()

I am about to raise an exception


Exception: This is always raised

In [13]:
def handler() -> None:
    try:
        never_returns()
        print("Never executed")
    except Exception as ex:
        print(f"I caught an exception: {ex!r}")
    print("Executed after the exception")



In [14]:
handler()

I am about to raise an exception
I caught an exception: Exception('This is always raised')
Executed after the exception


The `never_returns()` function happily informs us that it is about to raise an exception and raises it. 

The `handler()` function's except clause catches the exception. Once caught, we are able to clean up after ourselves (in this case, by outputting that we are handling the situation), and continue on our way. 

The remainder of the code in the `never_returns()` function remains unexecuted, but the code in the `handler()` function after the try: statement is able to recover and continue.

Note the indentation around `try and except`. The `try` clause wraps any code that might throw an exception. The `except` clause is then back on the same indentation level as the `try` line. Any code to handle the exception is indented inside the `except` clause. Then normal code resumes at the original indentation level.

The problem with the preceding code is that it uses the `Exception` class to match any type of exception. 

What if we were writing some code that could raise either `TypeError` or `ZeroDivisionError`? 

We might need to catch `ZeroDivisionError` because it reflects a known object state, but let any other exceptions propagate to the console because they reflect bugs we need to catch and kill. 

Can you guess the syntax?

In [15]:
from typing import Union

def funny_division(divisor:float) -> Union[str, float]:
    try:
        return 100 / divisor
    except ZeroDivisionError:
        return "Zero is not a good idea!"


This function does a simple computation. We've provided the type hint of float for the divisor parameter. We can provide an integer, and ordinary Python type coercion will work. 

The mypy tool is aware of the ways integers can be coerced to floats, saving it from having to obsess over the parameter types.

We do, however, have to be very clear about the return types. 

If we don't raise an exception, we'll compute and return a floating result. If we do raise a `ZeroDivisionError` exception, it will be handled, and we'll return a string result. 

Any other exceptions? Let's try it and see:

In [16]:
funny_division(0)

'Zero is not a good idea!'

In [19]:
print(funny_division(50.0))

2.0


In [18]:
print(funny_division("hello"))

TypeError: unsupported operand type(s) for /: 'int' and 'str'

 If we don't specify matching the `ZeroDivisionError` exception class, our handler would also see the `TypeError`, and accuse us of dividing by zero when we sent it a string, which is not a proper behavior at all.
 
Python also has a bare except syntax. Using `except`: with no exception class to match is widely frowned upon because it will prevent an application from simply crashing when it should. 

We generally use `except Exception`: to explicitly catch a sensible set of exceptions.

The bare except syntax is actually the same as using `except BaseException`:, which attempts to handle system-level exceptions that are often impossible to recover from. 

Indeed, this can make it impossible to crash your application when it's misbehaving.

We can even catch two or more different exceptions and handle them with the same code. 

Here's an example that raises three different types of exceptions. 

It handles TypeError and ZeroDivisionError with the same exception handler, but it may also raise a ValueError error if you supply the number 13:

In [20]:
def funny_division2(divisor:float) -> Union[str, float]:
    try:
        if divisor == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / divisor
    except (ZeroDivisionError, TypeError):
        return "Enter a number other than zero"

We've included multiple exception classes in the except clause. This lets us handle a variety of conditions with a common handler. 

Here's how we can test this with a bunch of different values:

In [22]:
for val in (0, "hello", 50.0, 13):
    print(f"Testing {val!r}:", end=" ")
    print(f"Result: {funny_division2(val)}")

Testing 0: Result: Enter a number other than zero
Testing 'hello': Result: Enter a number other than zero
Testing 50.0: Result: 2.0
Testing 13: 

ValueError: 13 is an unlucky number